In [ ]:
import data_loaders as dl
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
import pynwb
from pynwb import NWBHDF5IO, NWBFile, TimeSeries

In [ ]:
import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(0)

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.gridspec import GridSpec

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

color_names = ["windows blue",
               "red",
               "amber",
               "faded green",
               "dusty purple",
               "orange",
               "clay",
               "pink",
               "greyish",
               "mint",
               "light cyan",
               "steel blue",
               "forest green",
               "pastel purple",
               "salmon",
               "dark brown"]

colors = sns.xkcd_palette(color_names)

import ssm
from ssm.util import random_rotation
from ssm.plots import plot_dynamics_2d

In [ ]:
from sklearn.model_selection import train_test_split

## One file

In [ ]:
file_name='Data/sub-monk-g_ses-session4.nwb'#'Data/sub-monk-j_ses-session0.nwb'
newRate = 10#Hz
acqRate =60#Hz
nTrials = 62
use_trials=np.arange(nTrials)

input_dim = 4
state_dim = 4
testfrac = 0.1

In [ ]:
fig1 = plt.figure()
ax = fig1.add_subplot(111)
etrain = []
etest = []

for iter in range(5):
    np.random.seed()
    newTrials = np.random.permutation(nTrials)
    obs, inputs, pos = dl.getData_for_LDS(file_name, acqRate=acqRate, newRate=newRate, vis=True, use_trials=newTrials )

    ## split into train test:
    obs_train, obs_test, inp_train, inp_test = train_test_split( obs, inputs, test_size=testfrac)
    
    # Known paramters
    time_bins = obs.shape[0]
    obs_dim = obs.shape[1]

    ### fit on training data:
    lds_inp = ssm.LDS(obs_dim, state_dim, M=input_dim, emissions="poisson")
    elbos, q = lds_inp.fit((obs_train).astype(int), inputs=inp_train, num_iters=20)
    etrain.append(elbos[-1]/(obs_train.shape[0]*obs_train.shape[1]))


    # eigenvalues of dynamics:
    A = lds_inp.dynamics.A
    eigenvalues, eigenvectors = np.linalg.eig(A-np.eye(A.shape[0]))
    ax.scatter( np.real(eigenvalues), np.imag(eigenvalues), alpha=0.3)

    elbos2, posterior = lds_inp.approximate_posterior( (obs_test).astype(int), inputs=inp_test )
    etest.append(elbos2[-1]/(obs_test.shape[0]*obs_test.shape[1]))



ax.plot([0,0],[-.1,.1],'k--')
